# 2021_05_03 Delay Analysis

Notebook to perform analysis on delay period (2 or 4 seconds) of PWM task recording from mPFC. Functions tested & debugged in event_centering.ipynb and moving here for clean start

### Libraries & Settings

In [1]:
# libraries
import sys; sys.path.insert(0, '..') # if you don't find it here, look one above
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as spio
import pickle
import warnings

from tqdm import tqdm
from scipy import stats
from scipy.ndimage import gaussian_filter1d
from io_utils import *
from plotting_utils import * 


# settings
%load_ext autoreload
%autoreload 2
sns.set_context("talk")

### Set paths & defaults

In [2]:
### Set paths & defaults

sess_names = np.array(['data_sdb_20190724_193007_fromSD', 'data_sdc_20190902_145404_fromSD', 'W122_07_27_2019_1_fromSD', 
            'W122_08_06_2019_1_fromSD', 'W122_08_07_2019_1_fromSD', 'W122_08_31_2019_1_fromSD', 'W122_09_04_2019_1_fromSD',
            'W122_09_09_2019_1_fromSD', 'W122_09_12_2019_1_fromSD','W122_09_21_2019_1_fromSD', 'W122_09_24_2019_1_fromSD'])

base_path  = 'Y:\jbreda\ephys\post_sort_analysis\sorted_pre_bdata' 
beh_mat   = 'protocol_info.mat'
spks_mat  = 'ksphy_clusters_foranalysis.mat'


# create paths
sess_paths = [os.path.join(base_path, sess) for sess in sess_names]
beh_paths  = [os.path.join(path, beh_mat) for path in sess_paths]
spks_paths = [os.path.join(path, spks_mat) for path in sess_paths]

# alginment info (hard coded here & in fx)
names = ['aud1on', 'aud1off', 'aud2on', 'aud2off', 'trial_all', 'delay_overlap', 
         'delay2s', 'trial2s', 'delay4s', 'trial4s']
windows = [[-400, 600], [-500, 500], [-400, 600], [-500, 500], [-100, 5000], [-300, 2300],
           [-150, 2150],[-200, 2400], [-150, 4150], [-200, 4400]]

align_windows = dict(zip(names,windows))

### Load in data & align spikes to events - 2 and 4 second delay period

I'm filtering the dataframe below specifically for delay analysis (no psychometric sounds, no violations, no 6 second trials)

In [ ]:
events = ['delay2s', 'delay4s']
fig_save_path = os.path.join(os.getcwd(), 'figures', 'delay_loudness', '2s_4s_separate')

for name, path, behp, spksp in zip(sess_names, sess_paths, beh_paths, spks_paths):
    
    print(f"Loading session {name}")

    beh_df, spks_dict = load_and_wrangle(behp, spksp, overwrite=False)
    
    delay_analysis_df = beh_df[(beh_df['delay'] != 6) & (beh_df['first_sound'] != 'psycho') &
                                     (beh_df['hit_hist'] != 'viol')]
    
    del_2_df = delay_analysis_df[(delay_analysis_df['delay'] == 2)] 
    del_4_df = delay_analysis_df[(delay_analysis_df['delay'] == 4)]
    dfs = [del_2_df, del_4_df]
    
    print("Aligning to events")
    
    sess_aligned = event_align_session(spks_dict, delay_analysis_df, path, overwrite=True, 
                                   file_name='delay_analysis_aligned_spks_24_split.pkl')
    
    print("Plotting")
    
    run_delay_analysis_for_session(name, sess_aligned, align_windows, events, 
                                   dfs, fig_save_path, path)
    

### Load in data & align spikes to events- overlapping 2s of both delay periods

The figures above appeared to have no affect of first sound loudness on firing rate. In going back to the fiugres from last meeting, I realized I was only doing hit trials, and I collapsed the 2 and 4 s trials together and only looked at the first 2000 ms of delay. So, I will try to repeate this.

In [ ]:
events = ['delay_overlap']
fig_save_path = os.path.join(os.getcwd(), 'figures', 'delay_loudness', 'overlap')
for name, path, behp, spksp in zip(sess_names, sess_paths, beh_paths, spks_paths):
    
    print(f"Loading session {name}")
  
    beh_df, spks_dict = load_and_wrangle(behp, spksp, overwrite=False)
    
    delay_analysis_df = beh_df[(beh_df['delay'] != 6) & (beh_df['first_sound'] != 'psycho') &
                                     (beh_df['hit_hist'] == 'hit')]
    
    dfs = [delay_analysis_df] # keeping the same structure as above
    
    print("Aligning to events")
    
    sess_aligned = event_align_session(spks_dict, delay_analysis_df, path, overwrite=False, 
                                   file_name='delay_analysis_aligned_spks_24_together.pkl')
    
    print("Plotting")
    
    # Note, switched the window here for regression to be [300:-300] becuase of how
    # alignmnet was performed (currently line 399 in plotting utils)
    run_delay_analysis_for_session(name, sess_aligned, align_windows, events, 
                                   dfs, fig_save_path, path)
    

### Analyze all delay periods, extend the window to 600 ms, change convlution to zeros

Wrap was not the correct way to deal with edges. Setting to constant for now, and extending the area larger (from 300 to 600 ms) to make sure they really don't influence delay period. Plot psth will have xlim -100 + 2100 though.

In [6]:
events = ['delay_overlap']
fig_save_path = os.path.join(os.getcwd(), 'figures', 'delay_loudness', 'overlap246')
for name, path, behp, spksp in zip(sess_names, sess_paths, beh_paths, spks_paths):
    
    print(f"Loading session {name}")
  
    beh_df, spks_dict = load_and_wrangle(behp, spksp, overwrite=False)
    
    print
    
    delay_analysis_df = beh_df[(beh_df['first_sound'] != 'psycho') &
                                     (beh_df['hit_hist'] == 'hit')]
    
    dfs = [delay_analysis_df] # keeping the same structure as above
    
    print("Aligning to events")
    
    sess_aligned = event_align_session(spks_dict, delay_analysis_df, path, overwrite=True, 
                                   file_name='delay_analysis_aligned_spks_246_together.pkl')
    
    print("Plotting")
    
    # Note, switched the window here for regression to be [300:-300] becuase of how
    # alignmnet was performed (currently line 399 in plotting utils)
    run_delay_analysis_for_session(name, sess_aligned, align_windows, events, 
                                   dfs, fig_save_path, path)
    

Loading session data_sdb_20190724_193007_fromSD
Aligning to events
no file found, running alignment for session
Plotting


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:177: RuntimeWarning: Mean of empty slice
  smoothed_mean = np.nanmean(np.array(smoothed_trials), axis=0)


Loading session data_sdc_20190902_145404_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_07_27_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:177: RuntimeWarning: Mean of empty slice
  smoothed_mean = np.nanmean(np.array(smoothed_trials), axis=0)


Loading session W122_08_06_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_08_07_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_08_31_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:177: RuntimeWarning: Mean of empty slice
  smoothed_mean = np.nanmean(np.array(smoothed_trials), axis=0)


Loading session W122_09_04_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_09_09_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting


C:\Users\JB\anaconda3\envs\PWM_ephys\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\JB\anaconda3\envs\PWM_ephys\lib\site-packages\numpy\core\_methods.py:224: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\JB\anaconda3\envs\PWM_ephys\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\JB\anaconda3\envs\PWM_ephys\lib\site-packages\numpy\core\_methods.py:224: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:177: RuntimeWarning: Mean of empty slice
  smoothed_mean = np.nanmean(np.array(smoothed_trials), axis=0)


Loading session W122_09_12_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_09_21_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_09_24_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting


Ideas/TODO:
* what if filtered only by the middle sounds that are different?
* final functions need documentation
* update README (DO IT FOR YOURSELF)
* change pvalue to display more #s
* try plotting together on same plot
* make a stimulus plot (X,Y)
* jitter the scatter plot